In [ ]:
from pytorch_detection_tools import get_predictions, visualize_predictions
from training_tools import create_ssd_model
import torch

model_name = "exp_ssd.pth"
model_state_path = f"../trained_models/{model_name}"
model = create_ssd_model(num_classes=9)
checkpoint = torch.load(model_state_path)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()
image_path = "../ozm72_3.jpg"
device = torch.device('cpu')
boxes, labels, scores = get_predictions(model, image_path, device, threshold=0.2)
visualize_predictions(image_path, boxes, labels, scores)

In [ ]:
from torchvision.models.detection import ssd
model = ssd.ssd300_vgg16(pretrained=True)
model.eval()
image_path = "../traffic.jpg"
boxes, labels, scores = get_predictions(model, image_path, torch.device('cpu'), threshold=0.65)
visualize_predictions(image_path, boxes, labels, scores, coco=True)

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from training_tools import collate_fn, get_custom_dataset, save_model, create_ssd_model
import os
import json

model_state_path = '../trained_models/exp_ssd.pth'

print("Loading dataset...")
train_data_dir = "../explosives_small_dataset/coco/train"

my_dataset = get_custom_dataset(train_data_dir)
print('Number of samples: ', len(my_dataset))
# DataLoader instance
data_loader = DataLoader(my_dataset,
                         batch_size=2,
                         shuffle=True,
                         collate_fn=collate_fn)

# Initialize the model
print("Loading the model...")
num_classes = 9
model = create_ssd_model(num_classes)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params=params, lr=0.00001, momentum=0.9)
lr_scheduler = StepLR(optimizer, step_size=2)
epoch = 0
model.train()
i = 0
for images, targets in data_loader:
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    optimizer.zero_grad()
    losses = model(images, targets)
    loss = sum(loss for loss in losses.values())
    loss.backward()
    optimizer.step()
    if not i % 2:
        print(f"Epoch: {epoch}, Iteration: {i}, Loss: {loss.item()}")
    i += 1
lr_scheduler.step()
epoch+=1
save_model(model, model_state_path, optimizer, epoch, loss)